# Calculate damages

Finally, we use the extracted flood and network data to perform an expected annual damages calculation

In [ ]:
from glob import glob
import os

import geopandas as gpd
import pandas as pd
from pyproj import Geod
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import rasterio
from scipy.integrate import simpson, cumulative_trapezoid

from utils import aqueduct_rp
from rasterise import (
    check_raster_grid_consistent,
    split_linestrings,
    cell_indices_assigner,
    raster_lookup,
)

In [ ]:
data_dir = "data"
country_iso = "bgd"

In [ ]:
# flood hazard data to use, pulled from the autopkg API
epoch = 2050
scenario = "rcp4p5"
raster_paths = glob(f"data/{country_iso}/wri_aqueduct*/*{scenario}_wtsub_{epoch}*.tif")
raster_paths = sorted(raster_paths, key=aqueduct_rp, reverse=True)

(network_file,) = glob(f"data/{country_iso}/gri_osm*/*.gpkg")
network = gpd.read_file(network_file)

In [ ]:
# filter network to linestrings only (edges)
lines = network[network.geometry.type == "LineString"]

# filter to roads
desired_layers = {"motorway", "primary", "secondary", "tertiary", "trunk"}
lines = lines[lines.asset_type.isin(desired_layers)]

# error if grids not consistent
check_raster_grid_consistent(raster_paths)

# split edges on raster grid
raster_path, *other_raster_paths = raster_paths
raster = rasterio.open(raster_path)
splits = split_linestrings(lines, raster)

# calculate split edge lengths
geod = Geod(ellps="WGS84")
meters_per_km = 1_000
splits["length_km"] = splits.geometry.apply(geod.geometry_length) / meters_per_km

# which cell is each split edge in?
assigner = cell_indices_assigner(raster)
raster_indices = splits.geometry.apply(assigner)

# join raster indices to geometries with shared index
splits_with_raster_indices = splits.join(raster_indices)

In [ ]:
# map raster indices as visual check
f, (ax_i, ax_j) = plt.subplots(ncols=2)

ax_i = splits_with_raster_indices.plot(ax=ax_i, column="raster_i", cmap="Reds")
ax_i.set_title("raster_i")

splits_with_raster_indices.plot(ax=ax_j, column="raster_j", cmap="Blues")
ax_j.set_title("raster_j")
f.tight_layout()

In [ ]:
for path in raster_paths:
    splits_with_raster_indices[f"rp-{aqueduct_rp(path)}"] = raster_lookup(
        splits_with_raster_indices, path
    )

hazard_intensities = splits_with_raster_indices
hazard_intensities.describe()

In [ ]:
def logistic_min(
    x: float | np.ndarray, L: float, m: float, k: float, x_0: float
) -> float | np.ndarray:
    """
    Logistic function with a minimum value, m.

    Args:
        x: Input values
        L: Maximum output value
        m: Minimum output value
        k: Steepness parameter
        x_0: Location of sigmoid centre in x

    Returns:
        Output values
    """

    return m + (L - m) / (1 + np.exp(-k * (x - x_0)))


# define a damage function
damage_curve = lambda x: logistic_min(x, 1, 0, 2, 2)

# have a look at it
f, ax = plt.subplots()
x = np.linspace(0, 5, 20)
ax.plot(x, damage_curve(x), ".", ls="-")
ax.set_xlabel("Flood depth [meters]")
ax.set_ylabel("Damage fraction")
ax.set_title("Damage function")
ax.grid()

In [ ]:
# calculate how badly each split edge is damaged by the flooding
damage_fractions = hazard_intensities.copy()
hazard_cols = [col for col in hazard_intensities.columns if col.startswith("rp-")]
damage_fractions[hazard_cols] = damage_fractions[hazard_cols].applymap(damage_curve)

In [ ]:
# calculate the cost of damage
reconstruction_cost_currency_per_km = 1e5

damage_cost = damage_fractions.copy()
for col in hazard_cols:
    damage_cost[col] = (
        damage_cost[col] * damage_cost.length_km * reconstruction_cost_currency_per_km
    )

grouped_damage_cost = damage_cost[hazard_cols].groupby(damage_cost.original_index).sum()
probability_per_year = 1 / np.array(
    [int(col.replace("rp-", "")) for col in hazard_cols]
)

damage_probability_curve = grouped_damage_cost.copy()
damage_probability_curve.columns = probability_per_year

In [ ]:
# plot the damage-probability curve
f, ax = plt.subplots()
damage_probability = damage_probability_curve.sum()
ax.plot(damage_probability.index, damage_probability.values, ".", ls="-")
ax.grid()
ax.set_xlabel("Probability per given year")
ax.set_ylabel("Damage cost [currency]")
ax.set_title(f"Damage-probability curve\n{scenario.upper()} {epoch}")

In [ ]:
# check how the damage cost cumulatively grows as a function of probability
y = cumulative_trapezoid(grouped_damage_cost.sum(), probability_per_year)
f, ax = plt.subplots()
ax.plot(probability_per_year[1:], 100 * (y / y[-1]), ".", ls="-")
ax.set_xlabel("Probability per given year")
ax.set_ylabel("Cumulative expected damages [% total]")
ax.set_title("Cumulative Expected Damages")
# ax.set_xscale("log")
ax.grid()

# here, the tail risks aren't adding much to the expected annual damage figure

In [ ]:
# calculate the expected annual damages for every edge
# that is, integrate the damage-probability curve, for every row
EAD = lines[["geometry"]].copy()
EAD["ead"] = simpson(grouped_damage_cost, x=probability_per_year, axis=1)

# map the expected annual damages
f, ax = plt.subplots(figsize=(10, 10))

border = gpd.read_file(os.path.join(data_dir, country_iso, "territory.gpkg"))
border.plot(ax=ax, alpha=0.1, color="black")

EAD.plot(
    ax=ax,
    column="ead",
    legend=True,
    cmap="RdPu",
    norm=matplotlib.colors.LogNorm(vmin=1e0, vmax=EAD.ead.max()),
)
ax.grid()
ax.set_title(f"Expected Damages [currency per annum]\nTotal: {EAD.ead.sum():.2E}")